In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
# PYTEST_XFAIL_AND_SKIP_NEXT: ROL and fireshape are only expected to work with real numbers
assert arch == "real"

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import firedrake
else:
    try:
        import firedrake
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        import firedrake  # noqa: F401

In [ ]:
%load_ext wurlitzer

In [ ]:
# Adapted from https://bitbucket.org/pyrol/pyrol/src/master/tests/test_numpy_vector.py

In [ ]:
import ROL
import ROL.numpy_vector

In [ ]:
x = ROL.numpy_vector.NumpyVector(2)
x.data[0] = 0.5
x.data[1] = 0.5
y = ROL.numpy_vector.NumpyVector(2)
y.data[0] = 0.2
y.data[1] = 0.3
z = ROL.numpy_vector.NumpyVector(2)
z.data[0] = 0.1
z.data[1] = 0.7
"""
u = x.checkVector(y, z)
assert sum(u) < 1e-12
"""

In [ ]:
class MyObj(ROL.Objective):
    def __init__(self):
        ROL.Objective.__init__(self)

    def value(self, x, tol):
        return (x[0] - 1)**2 + x[1]**2

    def gradient(self, g, x, tol):
        g[0] = 2 * (x[0] - 1)
        g[1] = 2 * x[1]

In [ ]:
paramsDict = {
    "Step": {
        "Line Search": {
            "Descent Method": {
                "Type": "Quasi-Newton Method"
            }
        },
        "Type": "Interior Point",
        "Interior Point": {
            "Initial Barrier Penalty": 0.01,
            "Barrier Penalty Reduction Factor": 0.15,
            "Minimum Barrier Penalty": 1e-8
        },
        "Composite Step": {
            "Output Level": 0
        }
    },
    "Status Test": {
        "Gradient Tolerance": 1e-12,
        "Step Tolerance": 1e-16,
        "Constraint Tolerance": 1e-12,
        "Iteration Limit": 10
    }
}

In [ ]:
obj = MyObj()
params = ROL.ParameterList(paramsDict, "Parameters")
x = ROL.numpy_vector.NumpyVector(2)
optimProblem = ROL.OptimizationProblem(obj, x)
solver = ROL.OptimizationSolver(optimProblem, params)
solver.solve()
assert round(x[0] - 1.0, 6) == 0.0
assert round(x[1], 6) == 0.0